In [40]:
import pandas as pd
import numpy as np

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.corpus import stopwords

from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS, TfidfVectorizer

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora, models

> organize the text into two topics, plot cosine similarities betwen vectorized keywords

In [41]:
text = pd.Series([
    "Stretched Growth Story Casts a Shadow on AAPL Stock",
    "Apple (NASDAQ:AAPL) Earning Somewhat Favorable Media Coverage, Analysis Shows",
    "Flat Affect: Markets Tread Water With Fed, Other Central Bank Meetings Ahead",
    "Yes, Apple Stock Can Make New Highs This Year",
    "U.S. Investment Analyst is Upbeat on Apple's upcoming Q3 Financials, citing strong service revenues & Growth in China",
    "Apple (AAPL) Outpaces Stock Market Gains: What You Should Know",
    "In Case The Bears Are Right About Apple",
    "Microsoft says its Teams app is bigger than Slack and growing faster",
    "The Slackification of the American Home",
    "Here are the biggest analyst calls of the day: Slack, Zoom, Morgan Stanley & more",
    "If Slack is so good, why are so many companies trying to fix it?"
])

In [42]:
def download_packages():
    import nltk
    import ssl

    try:
        _create_unverified_https_context = ssl._create_unverified_context
    except AttributeError:
        pass
    else:
        ssl._create_default_https_context = _create_unverified_https_context

    nltk.download("wordnet")

In [43]:
download_packages()

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/pansproperties/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [44]:
CUSTOM_STOPWORDS = ENGLISH_STOP_WORDS.union(
    ["like"] # add more stopwords here as needed
)

def stem_func(text):
    stemmer = SnowballStemmer('english')
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def is_meaningful(token):
    cond1 = token not in CUSTOM_STOPWORDS
    cond2 = len(token) > 3
    return cond1 and cond2

def topic_prepare(text):
    result, tokens = [], gensim.utils.simple_preprocess(text)
    for token in tokens:
        if is_meaningful(token):
            result.append(stem_func(token))
            #result.append(token)
    return result

In [45]:
text = text.apply(topic_prepare)

In [46]:
dictionary = gensim.corpora.Dictionary(text)
dictionary.filter_extremes(no_below=1, no_above=0.5, keep_n=1000)
bow_dict = [dictionary.doc2bow(doc) for doc in text]

In [47]:
tfidf = models.TfidfModel(bow_dict)
corpus_tfidf = tfidf[bow_dict]
lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics = 5, id2word=dictionary, passes=2, workers=2)

In [48]:
for idx, topic in lda_model.print_topics(-1):
    print("topic {0} words {1} \n".format(idx, topic))

topic 0 words 0.054*"case" + 0.053*"bear" + 0.053*"right" + 0.027*"appl" + 0.014*"home" + 0.014*"slackif" + 0.014*"american" + 0.014*"slack" + 0.014*"stock" + 0.014*"good" 

topic 1 words 0.039*"american" + 0.039*"slackif" + 0.039*"home" + 0.031*"call" + 0.031*"morgan" + 0.031*"zoom" + 0.031*"stanley" + 0.031*"biggest" + 0.028*"media" + 0.028*"favor" 

topic 2 words 0.044*"market" + 0.038*"outpac" + 0.038*"gain" + 0.038*"know" + 0.030*"water" + 0.030*"affect" + 0.030*"flat" + 0.030*"bank" + 0.030*"meet" + 0.030*"central" 

topic 3 words 0.043*"year" + 0.043*"high" + 0.043*"make" + 0.035*"microsoft" + 0.035*"team" + 0.035*"faster" + 0.035*"grow" + 0.035*"say" + 0.035*"bigger" + 0.029*"stock" 

topic 4 words 0.037*"compani" + 0.037*"tri" + 0.037*"good" + 0.036*"growth" + 0.032*"shadow" + 0.032*"stori" + 0.032*"stretch" + 0.032*"cast" + 0.025*"china" + 0.025*"servic" 

